## Введение в обработку естественного языка

Домашнее задание №9

Урок 9. Языковое моделирование

*Формат именования файла домашней работы: FIO_NLP_HW_N.ipynb, где N - номер домашнего задания*

**

###Задание  
Разобраться с моделью генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

**Установка библиотек**

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
import typing
import time
import requests
import bs4
from urllib.parse import urljoin
from datetime import datetime
import tqdm
import shutil

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0"
    }

url = 'http://slushat-tekst-pesni.ru/stas-mihailov/'

In [4]:
# парсим тексты песен стаса михайлова

response = requests.get(url, headers=headers)
soup = bs4.BeautifulSoup(response.text, 'lxml')
song_list = soup.find_all('ul', attrs={'class', 'nolist'})

In [5]:
all_songs = []

for part in song_list:
    all_songs = all_songs + part.find_all('li')

In [6]:
song_urls = [urljoin(response.url, itm.find('a').attrs.get('href')) for itm in all_songs if itm]

In [7]:
song_texts = ''

for url in tqdm.tqdm(song_urls):
    time.sleep(.5)
    response = requests.get(url, headers=headers)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    song_texts += soup.find('div', attrs={'class': 'abp'}).text

100%|██████████| 130/130 [02:59<00:00,  1.38s/it]


In [8]:
print('Length of text: {} characters'.format(len(song_texts)))

Length of text: 103924 characters


In [9]:
song_texts = song_texts + song_texts

In [10]:
open("stas.txt", "w").write(song_texts)

207848

In [11]:
song_texts = open("stas.txt", 'r').read()

In [12]:
# The unique characters in the file
vocab = sorted(set(song_texts))
print('{} unique characters'.format(len(vocab)))

108 unique characters


In [13]:
song_texts[:50]

'Сколько ты со мною испытала мама,Перед тем, как вс'

In [14]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in song_texts])

In [15]:
text_as_int

array([ 63,  82,  86, ...,  86, 103,   8])

In [16]:
text_as_int, song_texts, len(text_as_int), len(song_texts)

(array([ 63,  82,  86, ...,  86, 103,   8]),
 'Сколько ты со мною испытала мама,Перед тем, как встал на ноги я.Сколько ты ночей бессонных простояла мама,У кроватки детской не спала.У кроватки детской не спала.Моя милая мама,Свет твоих глаз всюду рядом со мной.Бреду по жизни бродяга,Ты от беды меня, любимая укрой.Не старей, прошу тебя, родная мама.Оставайся вечно молодой.Пусть сегодня не с тобой, но, ты слышишь мама,Я вернусь к тебе одной родной.Моя милая мама,Свет твоих глаз всюду рядом со мной.Бреду по жизни бродяга,Ты от беды меня, любимая, укрой.Моя милая мама,Свет твоих глаз всюду рядом со мной.Бреду по жизни бродяга,Ты от беды меня, любимая, укрой.Моя милая мамаСвет твоих глаз всюду рядом со мной.Бреду по жизни бродяга,Ты от беды меня, любимая, укрой.Ты от беды меня, любимая, укрой.Ты своей любовью, от беды меня укрой.Ты прости меня родная Что творю я сам не знаю Просто очень плохо без тебя Незнакомые все лица А душе моей не спиться Дни я проколачиваю зря Незнакомые все лица А душ

### train and target

In [17]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(song_texts)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

С
к
о
л
ь


In [18]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Сколько ты со мною испытала мама,Перед тем, как встал на ноги я.Сколько ты ночей бессонных простояла '
'мама,У кроватки детской не спала.У кроватки детской не спала.Моя милая мама,Свет твоих глаз всюду ряд'
'ом со мной.Бреду по жизни бродяга,Ты от беды меня, любимая укрой.Не старей, прошу тебя, родная мама.О'
'ставайся вечно молодой.Пусть сегодня не с тобой, но, ты слышишь мама,Я вернусь к тебе одной родной.Мо'
'я милая мама,Свет твоих глаз всюду рядом со мной.Бреду по жизни бродяга,Ты от беды меня, любимая, укр'


In [19]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [20]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Сколько ты со мною испытала мама,Перед тем, как встал на ноги я.Сколько ты ночей бессонных простояла'
Target data: 'колько ты со мною испытала мама,Перед тем, как встал на ноги я.Сколько ты ночей бессонных простояла '


In [21]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [22]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [23]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True, 
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [24]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 108) # (batch_size, sequence_length, vocab_size)


In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           27648     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 108)           110700    
                                                                 
Total params: 30,563,436
Trainable params: 30,563,436
No

In [27]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 108), dtype=float32, numpy=
array([[ 4.71196745e-06, -1.05666131e-05,  5.14303701e-06, ...,
        -5.60105309e-06, -7.83947871e-06,  1.51037166e-05],
       [ 1.15626499e-05, -5.43051465e-05,  2.79264823e-05, ...,
        -1.60996969e-05, -3.96487958e-05,  4.19688404e-05],
       [ 2.01838702e-05, -1.07536929e-04,  6.42051600e-05, ...,
        -2.93146568e-05, -1.03910075e-04,  6.73554896e-05],
       ...,
       [ 1.46514503e-03, -8.67445546e-04, -1.83362700e-03, ...,
         4.61277086e-05, -4.81777126e-04, -8.50867946e-05],
       [ 1.55402615e-03, -7.42581673e-04, -1.80388859e-03, ...,
        -1.35161448e-04, -4.43069381e-04, -1.47711020e-04],
       [ 1.65264786e-03, -6.03156164e-04, -1.72266946e-03, ...,
        -3.06332950e-04, -4.22045472e-04, -2.00027600e-04]], dtype=float32)>

In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [29]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'е вернетесь, тревожа лишь память...Ушла не пол пути расстались непонятноИ нестерпимо боль измучила в'

Next Char Predictions: 
 '2ш0:o?:T5e–дНз2rб.—ьхИ!эпЗО5К"tмSщl355SяГ…\nаzУямв;(Н/4–МБ)р3 piгетg-Лч.][тМрdu0ьС-пЦovaМkьсnшмзпёс.Б'


Пока получается криво

## Train the model

In [30]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 108)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.682153


In [31]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [32]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

### Execute the training

In [33]:
EPOCHS = 120 # можно больше поставить чтобы генерировала достаточно хорпошо

In [34]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/120
32/32 [==============================] - 18s 343ms/step - loss: 3.5969
Epoch 2/120
32/32 [==============================] - 11s 342ms/step - loss: 3.3325
Epoch 3/120
32/32 [==============================] - 12s 341ms/step - loss: 3.1931
Epoch 4/120
32/32 [==============================] - 11s 343ms/step - loss: 2.9113
Epoch 5/120
32/32 [==============================] - 11s 340ms/step - loss: 2.6732
Epoch 6/120
32/32 [==============================] - 11s 341ms/step - loss: 2.5183
Epoch 7/120
32/32 [==============================] - 12s 357ms/step - loss: 2.3377
Epoch 8/120
32/32 [==============================] - 12s 351ms/step - loss: 2.1067
Epoch 9/120
32/32 [==============================] - 11s 345ms/step - loss: 1.8210
Epoch 10/120
32/32 [==============================] - 11s 347ms/step - loss: 1.5083
Epoch 11/120
32/32 [==============================] - 12s 353ms/step - loss: 1.1948
Epoch 12/120
32/32 [==============================] - 12s 366ms/step - loss: 0.9024
E

## Generate text

In [35]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_110'

In [36]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) # переазгрузили модель в батч-сайз 1 (1-01 в лекции)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            27648     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 108)            110700    
                                                                 
Total params: 30,563,436
Trainable params: 30,563,436


In [38]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [59]:
text_ = generate_text(model, start_string=u"я сдам сессию!")
print(text_)

я сдам сессию!
Назад не вернетесь, тревожа лишь память мою.
 Уходите, Вы часто без спроса уходите.
С улыбкой прощальной Вы в дата без тебя,
Так нужна ты мне, любимая моя.
Так нужна ты мне, любимая моя.Холодные глаза и в них застывший свет.
И горький крик разрежет тишину...Ты не жалел себя, не зная свой пределТы все света чистота.Лети, душа, лети, душа, скорей,От суеты от зависти людей.Лети, душа, и тронь облака,Где вечная любовь, где света чистота.Лети, лети, лети, лети, душа!Я так странница, Теряет дни, по с


In [60]:
len(text_)

514

In [61]:
# не идеально, но рифма есть)